# Introduction

CoHDL is a Python based hardware definition language. It turns Python code into VHDL by inspecting and translating the abstract syntax tree. The core of the language is similar to VHDL. Designs are made up of concurrent and sequential contexts (processes) and use signals/variables to store state and communicate with each other.

The following examples give a quick overview over what CoHDL designs look like. The used functions and language constructs are described in more detail in the other notebooks in this directory.

---
### Example 1

This first example defines an inverter circuit. It takes a single bit input and produces the inverse value on the output port.

In [ ]:
from cohdl import Entity, Port, Bit
from cohdl import std

class Inverter(Entity):
    input = Port.input(Bit)
    output = Port.output(Bit)

    def architecture(self):
        @std.concurrent
        def logic():
            self.output <<= ~self.input

The parts of this design are described in the comments of the following code block.

In [ ]:
# the cohdl module provides magic builtin types
from cohdl import Entity, Port, Bit

# the std module contains convenience functions/classes that wrap cohdl builtins
from cohdl import std

# inheriting from cohdl.Entity turns the class Inverter
# into a synthesizable entity
class Inverter(Entity):
    # ports are defined as class members
    # using the methods input/output/inout of the cohdl builtin Port
    # the argument defines the type of the port
    input = Port.input(Bit)
    output = Port.output(Bit)

    # a method named 'architecture' is used to
    # define the entities behavior
    def architecture(self):
        # arbitrary python code

        # functions decorated with std.concurrent or std.sequential
        # are translated to a hdl representation
        @std.concurrent
        def logic():
            # synthesizable subset of python

            # the left shift assignment operator
            # is used for signal assignments
            self.output <<= ~self.input

# convert the entity to VHDL and print it to stdout
print(std.VhdlCompiler.to_string(Inverter))
# use VhdlCompiler.to_dir to write the resulting VHDL representation
# to the file system instead of stdout
# std.VhdlCompiler.to_dir(Inverter, "output_dir")


---
### Example 2

The second example demonstrates sequential logic by implementing a simple LED blinker design.

In [ ]:
from cohdl import Entity, Port, Bit, Unsigned, Signal
from cohdl import std

class Blink(Entity):
    clk = Port.input(Bit)
    led = Port.output(Bit)

    def architecture(self):
        MAX_COUNTER = 100_000_000
        counter = Signal[Unsigned[31:0]](0)

        # std.Clock is a helper type that wraps clock signals
        # when used like this the sequential function
        # is 'called' on every rising edge
        @std.sequential(std.Clock(self.clk))
        def blink_led():
            if counter == 0:
                # assigning to the .next property is equivalent
                # to the '<<=' operator
                counter.next = MAX_COUNTER
                # invert the led when counter reaches zero
                self.led <<= ~self.led
            else:
                counter.next = counter - 1

print(std.VhdlCompiler.to_string(Blink))

---
### Example 3

Coroutines are functions, that can be suspended and resumed. One of the core features of CoHDL is the ability to translate async/await coroutines into VHDL state machines. This translation process is deterministic and allows clock accurate descriptions of sequential logic. The following example demonstrates this by implementing a data transaction. The same mechanism can be used to abstract more complex interfaces such as AXI.

The example defines a class `Interface`. It wraps the data and hand shake signals of a simple interface and provides a method `receive`.
The Python keyword `async` marks the method as a coroutine. It implements the following logic:

* wait until the `valid` signal becomes high
* set the acknowledge signal high for one clock period (`^=` operator)
* return the current state of the `data` signal

The caller of `receive` can use the coroutine without knowing the details of the underlying interface.

In [ ]:
from cohdl import Entity, Port, Bit, BitVector, Temporary
from cohdl import std

class Interface:
    def __init__(self, valid, ack, data):
        self.valid = valid
        self.ack = ack
        self.data = data
    
    async def receive(self):
        await self.valid
        self.ack ^= True
        return Temporary(self.data)

class CoroutineExample(Entity):
    clk = Port.input(Bit)

    data_in = Port.input(BitVector[32])
    valid = Port.input(Bit)
    ack = Port.output(Bit, default=False)

    data_out = Port.output(BitVector[32])

    def architecture(self):
        interface = Interface(self.valid, self.ack, self.data_in)

        @std.sequential(std.Clock(self.clk))
        async def process_receive():
            # wait for new data and update the data_out port
            self.data_out <<= await interface.receive()
            # when the end of a coroutine is reached
            # execution continues from the start (after one clock cycle)

print(std.VhdlCompiler.to_string(CoroutineExample))